# read data

In [4]:
import pandas as pd

ratings = pd.read_csv('ratings.dat', header=None, sep='::', names=['uid', 'iid', 'val', 'tag'])
# ratings = ratings.drop(columns=['tag'])
ratings.head()

/tmp/ipykernel_1318085/1367597354.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('ratings.dat', header=None, sep='::', names=['uid', 'iid', 'val', 'tag'])


,uid,iid,val,tag
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
users = pd.read_csv('users.dat', header=None, sep='::', names=['uid', 'gender', 'age', 'occup', 'zip'], engine='python')
# np.save('user_gender', gender_dict)

# data info

In [5]:
print('Number of ratings:', len(ratings))
print('Number of users:', len(ratings.uid.unique()))
print('Number of movies:', len(ratings.iid.unique()))

print(max(ratings.uid.unique()))

Number of ratings: 1000209
Number of users: 6040
Number of movies: 3706
6040


In [8]:
print('male:', len(users[users.gender == 'M']))
print('female:', len(users[users.gender == 'F']))

male: 4331
female: 1709


In [9]:
print(ratings.groupby('uid').agg('count'))

      iid  val  tag
uid                
1      53   53   53
2     129  129  129
3      51   51   51
4      21   21   21
5     198  198  198
...   ...  ...  ...
6036  888  888  888
6037  202  202  202
6038   20   20   20
6039  123  123  123
6040  341  341  341

[6040 rows x 3 columns]


# filter interaction < 5

In [10]:
print('user:', min(ratings.groupby('uid').agg('count').iid))
print('item:', min(ratings.groupby('iid').agg('count').uid))

user: 20
item: 1


In [11]:
from tqdm import tqdm

low = 5
while True:
    n_filter = 0
    # drop item
    print('>>> item:')
    user_data = ratings.groupby('iid').agg(list)
    for i in tqdm(ratings.iid.unique()):
        if len(user_data['uid'][i]) < low:
            ratings = ratings[ratings.iid.values != i]
            n_filter += 1

    # drop user
    print('>>> user:')
    item_data = ratings.groupby('uid').agg(list)
    for i in tqdm(ratings.uid.unique()):
        if len(item_data['iid'][i]) < low:
            ratings = ratings[ratings.uid.values != i]
            n_filter += 1
    
    # terminate
    if n_filter == 0:
        break

# reset index
ratings.reset_index(drop=True, inplace=True)

>>> item:


100%|██████████| 3706/3706 [00:03<00:00, 1056.55it/s] 


>>> user:


100%|██████████| 6040/6040 [00:00<00:00, 250047.83it/s]


>>> item:


100%|██████████| 3416/3416 [00:00<00:00, 224421.51it/s]


>>> user:


100%|██████████| 6040/6040 [00:00<00:00, 237706.74it/s]


In [12]:
print('user:', min(ratings.groupby('uid').agg('count').iid))
print('item:', min(ratings.groupby('iid').agg('count').uid))

user: 18
item: 5


# squeeze id

In [13]:
print(f'user id: {max(ratings.uid.unique())}/{len(ratings.uid.unique())}')
print(f'item id: {max(ratings.iid.unique())}/{len(ratings.iid.unique())}')

user id: 6040/6040
item id: 3952/3416


In [14]:
print(ratings)

         uid   iid  val        tag
0          1  1193    5  978300760
1          1   661    3  978302109
2          1   914    3  978301968
3          1  3408    4  978300275
4          1  2355    5  978824291
...      ...   ...  ...        ...
999606  6040  1091    1  956716541
999607  6040  1094    5  956704887
999608  6040   562    5  956704746
999609  6040  1096    4  956715648
999610  6040  1097    4  956715569

[999611 rows x 4 columns]


In [15]:
import numpy as np

# squeeze uid
ratings.uid -= 1

# squeeze iid
item_dict = {}
for i, ori in enumerate(ratings['iid'].unique()):
    item_dict[ori] = i
    
np.save('item_dict', item_dict)

ratings.iid = ratings.iid.map(item_dict)

# save file
ratings.sort_values(by=['uid', 'iid']).to_csv('ratings.csv', index=False)

In [16]:
import pandas as pd
new_pdr = pd.read_csv("ratings.csv")
new_pdr = new_pdr[['uid','iid','val']]

new_pdr['val'] = 1
print(new_pdr)

         uid   iid  val
0          0     0    1
1          0     1    1
2          0     2    1
3          0     3    1
4          0     4    1
...      ...   ...  ...
999606  6039  2893    1
999607  6039  2937    1
999608  6039  3018    1
999609  6039  3022    1
999610  6039  3344    1

[999611 rows x 3 columns]


In [17]:
from tqdm import tqdm
import time 
import pandas as pd
import random

train_ratio = 0.8
train_df = pd.DataFrame([], columns=['uid', 'iid', 'val'])
test_df = pd.DataFrame([], columns=['uid', 'iid', 'val'])

train_index = []
test_index = []


for i in range(len(new_pdr.uid.unique())):
    t1 = time.time()
    indices = new_pdr[new_pdr.uid == i].index
    index_list = indices.to_list()
    
    total = len(indices)

    n_train = int(total * train_ratio)
    
    train_idx = random.sample(index_list, n_train)
    test_idx = list(set(index_list) - set(train_idx))
    
    train_index.extend(train_idx)
    test_index.extend(test_idx)
    
    # if i%1000 == 0:
    #     print(i)

train_index = np.array(train_index)
test_index = np.array(test_index)

user_train = new_pdr.iloc[np.sort(train_index), :]
user_test = new_pdr.iloc[np.sort(test_index), :]

train_df = pd.concat([train_df,user_train], ignore_index=True)
test_df = pd.concat([test_df, user_test], ignore_index=True)  

print(train_df)
print(test_df)

# c=9/0
# train_df['val'] = np.float16(train_df['rating'])
# test_df['val'] = np.float16(test_df['rating'])

train_df[['uid', 'iid', 'val']].to_csv('train.csv', index=False)
test_df[['uid', 'iid', 'val']].to_csv('test.csv', index=False)

         uid   iid val
0          0     1   1
1          0     2   1
2          0     6   1
3          0     7   1
4          0     8   1
...      ...   ...  ..
797270  6039  2893   1
797271  6039  2937   1
797272  6039  3018   1
797273  6039  3022   1
797274  6039  3344   1

[797275 rows x 3 columns]
         uid   iid val
0          0     0   1
1          0     3   1
2          0     4   1
3          0     5   1
4          0    15   1
...      ...   ...  ..
202331  6039  2509   1
202332  6039  2568   1
202333  6039  2602   1
202334  6039  2624   1
202335  6039  2662   1

[202336 rows x 3 columns]


# divide train:test

In [18]:
print(f'user id: {max(ratings.uid.unique())}/{len(ratings.uid.unique())}')
print(f'item id: {max(ratings.iid.unique())}/{len(ratings.iid.unique())}')

user id: 6039/6040
item id: 3415/3416


In [21]:
import pandas as pd
ll = pd.read_csv("ratings.csv")
print(ll)

       uid   iid  val        tag
0        0     0    5  889751633
1        0     3    4  878543275
2        0     6    4  878542441
3        0     9    5  878543541
4        0    10    4  874965954
...    ...   ...  ...        ...
99282  942  1025    4  888640250
99283  942  1058    3  888640250
99284  942  1061    1  888640027
99285  942  1077    3  888640275
99286  942  1179    4  888639954

[99287 rows x 4 columns]


In [27]:
import pandas as pd
train_data = pd.read_csv("train.csv")
print(train_data)

         uid   iid  val
0          0     1    1
1          0     2    1
2          0     6    1
3          0     7    1
4          0     8    1
...      ...   ...  ...
797270  6039  2893    1
797271  6039  2937    1
797272  6039  3018    1
797273  6039  3022    1
797274  6039  3344    1

[797275 rows x 3 columns]


In [28]:
print(train_data.iloc[:, 0][0:30])

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
Name: uid, dtype: int64


In [24]:
user_num = train_data['uid'].max() + 1
item_num = train_data['iid'].max() + 1

# train_data = train_data.values.tolist()
train_data = train_data.values.tolist()

pos_dict = {}

for x in train_data:
    if x[0] not in pos_dict.keys():
        pos_dict[x[0]] = []
    pos_dict[x[0]].append(x[1])

np.save('pos_dict.npy', pos_dict)

In [56]:
d = np.load('pos_dict.npy', allow_pickle=True).item()
print(d[0])

[0, 6, 9, 10, 12, 13, 14, 22, 23, 24, 26, 29, 31, 32, 33, 34, 38, 39, 44, 46, 47, 49, 52, 53, 56, 64, 65, 71, 73, 75, 76, 77, 79, 83, 86, 88, 93, 94, 100, 106, 112, 113, 116, 117, 118, 122, 130, 139, 146, 147, 148, 156, 157, 166, 169, 170, 176, 177, 181, 182, 188, 191, 195, 199, 203, 208, 209, 215, 216, 217, 218, 220, 221, 224, 229, 233, 235, 239, 247, 250, 253, 254, 256, 260, 263, 269, 275, 277, 288, 292, 296, 297, 303, 305, 307, 311, 313, 317, 320, 321, 324, 328, 329, 330, 337, 338, 347, 351, 356, 357, 359, 363, 367, 371, 373, 379, 382, 385, 387, 390, 398, 402, 404, 407, 422, 431, 442, 443, 446, 450, 460, 465, 467, 469, 478, 489, 490, 497, 499, 500, 501, 504, 519, 520, 524, 525, 527, 532, 538, 541, 543, 544, 546, 549, 552, 553, 554, 563, 568, 574, 586, 588, 592, 595, 608, 609, 611, 619, 621, 642, 655, 658, 661, 681, 697, 712, 715, 716, 717, 725, 731, 748, 760, 765, 766, 768, 785, 793, 805, 808, 843, 848, 850, 851, 856, 858, 892, 910, 918, 922, 923, 926, 934, 963, 980, 984, 987, 1030,